In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import uproot
import pandas

In [2]:
class Samples:
    Signal_NonRes = -125
    Signal_Radion = [ -250, -260, -270, -280, -300, -320, -340, -350,
                      -400, -450, -500, -550, -600, -650, -750, -800, -900 ]
    Data = 0
    TT = 1
    DY = 2
    Wjets= 3
    SM_Higgs = 4
    other_bkg = 5

In [3]:
#path = "/gpfs/ddn/cms/user/cmsdas/2019/hh_bbtautau/anaTuples/"
path = "/Users/Tita/Desktop/Run2_analysis_hh_bbtautau/anaTuples/"
#path = "./anaTuples/"

samples = {'eTau': "eTau_tuple.root", 'muTau': "muTau_tuple.root", 'tauTau': "tauTau_tuple.root"} 
channel = "tauTau"

with uproot.open(path+samples[channel]) as file:
    tree = file[channel]
    df = tree.arrays('*', outputtype=pandas.DataFrame)

In [6]:
presel = (df.q_1 + df.q_2 == 0)
mX = -Samples.Signal_NonRes
n_s = np.sum(df[(df.sample_id == -mX) & presel].weight)
n_b = np.sum(df[(df.sample_id > 0) & presel].weight)

sigma = n_s / math.sqrt(n_s + n_b)
print('mX = {} GeV, sigma = {}'.format(mX, sigma))

mX = 125 GeV, sigma = 0.0
